# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
!pip install nltk

In [104]:
# import libraries

import numpy as np
import pandas as pd
import os
import sys
import pickle
import re
import nltk
from sqlalchemy import create_engine
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from scipy.stats import hmean
from scipy.stats.mstats import gmean

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download(['averaged_perceptron_tagger', 'wordnet', 'punkt'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mikhaildiazandrade/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mikhaildiazandrade/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mikhaildiazandrade/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
pd.set_option('display.max_columns', 500)

In [20]:
# load data from database
engine = create_engine('sqlite:///../data/disasterResponse.db')
df = pd.read_sql('DisasterResponse', engine)

df.head()

,id,message,original,genre,categories,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
df.shape

(26216, 41)

In [22]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
print(df.var())

id                        7.791397e+07
related                   1.894655e-01
request                   1.415400e-01
offer                     4.480979e-03
aid_related               2.426563e-01
medical_help              7.317702e-02
medical_products          4.757733e-02
search_and_rescue         2.685506e-02
security                  1.764402e-02
military                  3.172948e-02
child_alone               0.000000e+00
water                     5.971251e-02
food                      9.906904e-02
shelter                   8.047877e-02
clothing                  1.521050e-02
money                     2.250941e-02
missing_people            1.123832e-02
refugees                  3.226380e-02
death                     4.347204e-02
other_aid                 1.141726e-01
infrastructure_related    6.080918e-02
transport                 4.371467e-02
buildings                 4.826325e-02
electricity               1.988191e-02
tools                     6.028444e-03
hospitals                

In [25]:
#Remove column child alone 
df = df.drop(['child_alone'],axis=1)

In [60]:
#Create X and y variables for modelling
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre', 'categories'], axis=1)

### 2. Write a tokenization function to process your text data

In [61]:
def tokenize(text):
    """
    Tokenization
    Arguments:
        Text -> Text messages
    Output:
        Clean_tokens -> Tokens extracted from the provided texts messages
    """
    #Replace the urls with a urlplaceholder string
    url_rgx = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    #Extract the urls from the provided text 
    detected_urls = re.findall(url_rgx, text)
    
    #Replace url with a url placeholder string
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    #Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    #Clean tokens
    clean_tkns = []
    for tkn in tokens:
        clean_tkn = lemmatizer.lemmatize(tkn).lower().strip()
        clean_tkns.append(clean_tkn)
    return clean_tkns

In [ ]:
"""
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
"""

In [50]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [86]:
#
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))      
        ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])  
    return pipeline

#
def model_pipeline_2():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
            ('starting_verb_transformer', StartingVerbExtractor())
        ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [87]:
#Split and Train Data 
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = []
model = model_pipeline()
model_fitted = model.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [90]:
#predict on the test data
y_preds_tst = model_fitted.predict(X_test)

#classification report
def cls_report_table(y_test, y_preds_tst):
    for i, col in enumerate(y_test.columns): 
        print('-------------------------------------------------------')
        print("-->", col)
        print(classification_report(y_test.iloc[:,i], y_preds_tst[:,i]))

cls_report_table(y_test, y_preds_tst)

-------------------------------------------------------
--> related
              precision    recall  f1-score   support

           0       0.75      0.27      0.40      1538
           1       0.81      0.97      0.88      4968
           2       0.42      0.23      0.30        48

    accuracy                           0.80      6554
   macro avg       0.66      0.49      0.52      6554
weighted avg       0.79      0.80      0.76      6554

-------------------------------------------------------
--> request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5426
           1       0.90      0.42      0.58      1128

    accuracy                           0.89      6554
   macro avg       0.90      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

-------------------------------------------------------
--> offer
              precision    recall  f1-score   support

           0       1.00      1.00  

              precision    recall  f1-score   support

           0       0.87      0.97      0.91      4735
           1       0.89      0.61      0.72      1819

    accuracy                           0.87      6554
   macro avg       0.88      0.79      0.82      6554
weighted avg       0.87      0.87      0.86      6554

-------------------------------------------------------
--> floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6012
           1       0.90      0.37      0.52       542

    accuracy                           0.94      6554
   macro avg       0.93      0.68      0.75      6554
weighted avg       0.94      0.94      0.93      6554

-------------------------------------------------------
--> storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5938
           1       0.81      0.44      0.57       616

    accuracy                           0.94      

### 6. Improve your model
Use grid search to find better parameters. 

In [99]:
#
model_ = model_pipeline()
parameters = {
    'features__text_pipeline__count_vectorizer__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__count_vectorizer__max_df': (0.5, 1.0),
    'features__text_pipeline__count_vectorizer__max_features': (None, 20),
    'features__text_pipeline__tfidf_transformer__use_idf': (True, False),
#    'clf__n_estimators': [10, 100],
#    'clf__learning_rate': [0.01, 0.1],
#    'features__transformer_weights': (
#        {'text_pipeline': 1, 'starting_verb': 0.5},
#        {'text_pipeline': 0.5, 'starting_verb': 1},
#        {'text_pipeline': 0.8, 'starting_verb': 1},
#    )
}

cv = GridSearchCV(model_, param_grid= parameters, verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 77.2min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 137.2min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f8384879700>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'features__text_pipeline__count_vectorizer__max_df': (0.5,
                                                                               1.0),
                   

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [100]:
#predict on the test data
y_preds_tst2 = cv.predict(X_test)

#classification report
def cls_report_table(y_test, y_preds_tst2):
    for i, col in enumerate(y_test.columns): 
        print('-------------------------------------------------------')
        print("-->", col)
        print(classification_report(y_test.iloc[:,i], y_preds_tst2[:,i]))

cls_report_table(y_test, y_preds_tst2)

-------------------------------------------------------
--> related
              precision    recall  f1-score   support

           0       0.73      0.28      0.40      1538
           1       0.81      0.96      0.88      4968
           2       0.47      0.40      0.43        48

    accuracy                           0.80      6554
   macro avg       0.67      0.55      0.57      6554
weighted avg       0.79      0.80      0.76      6554

-------------------------------------------------------
--> request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5426
           1       0.90      0.45      0.60      1128

    accuracy                           0.90      6554
   macro avg       0.90      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

-------------------------------------------------------
--> offer
              precision    recall  f1-score   support

           0       1.00      1.00  

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4735
           1       0.89      0.58      0.70      1819

    accuracy                           0.86      6554
   macro avg       0.88      0.78      0.81      6554
weighted avg       0.87      0.86      0.85      6554

-------------------------------------------------------
--> floods
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6012
           1       0.91      0.31      0.46       542

    accuracy                           0.94      6554
   macro avg       0.93      0.65      0.71      6554
weighted avg       0.94      0.94      0.93      6554

-------------------------------------------------------
--> storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5938
           1       0.83      0.43      0.56       616

    accuracy                           0.94      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [105]:
xgboost_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(XGBClassifier())) 
    ])

a_p = xgboost_pipeline.fit(X_train, y_train)
y_pred_xgb= a_p.predict(X_test)
cls_report_table(y_test, y_pred_xgb)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:58:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:58:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:58:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:58:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objectiv

[19:00:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:00:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:01:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:01:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6246
           1       0.78      0.27      0.40       308

    accuracy                           0.96      6554
   macro avg       0.87      0.63      0.69      6554
weighted avg       0.96      0.96      0.95      6554

-------------------------------------------------------
--> buildings
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      6230
           1       0.74      0.41      0.52       324

    accuracy                           0.96      6554
   macro avg       0.85      0.70      0.75      6554
weighted avg       0.96      0.96      0.96      6554

-------------------------------------------------------
--> electricity
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6412
           1       0.57      0.31      0.40       142

    accuracy                           0

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [107]:
#saved the model
file_name_ = 'classifier.pkl' #sav 
pickle.dump(model, open(file_name_, 'wb'))

In [108]:
# load the model from disk
loaded_model = pickle.load(open('classifier.pkl', 'rb'))
loaded_model_= loaded_model.score(X_test, y_test)
print(loaded_model_)

0.24259993896856882


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.